# Patch daily with jura

In [1]:
import os
import numpy as np
from astropy.table import Table, join
from astropy.io import fits
# from desispec.io import read_table
from desispec.io.meta import faflavor2program
from specprodDB.util import cameraid

In [2]:
daily_tiles_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'daily', 'tiles-daily.csv')
daily_exposures_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'daily', 'exposures-daily.fits')
jura_tiles_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'jura', 'tiles-jura.csv')
jura_exposures_file = os.path.join(os.environ['DESI_SPECTRO_REDUX'], 'jura', 'exposures-jura.fits')
daily_tiles = Table.read(daily_tiles_file, format='ascii.csv')
daily_exposures = Table.read(daily_exposures_file, format='fits', hdu='EXPOSURES')
# daily_exposures = read_table(daily_exposures_file, ext='EXPOSURES')
daily_frames = Table.read(daily_exposures_file, format='fits', hdu='FRAMES')
# daily_frames = read_table(daily_exposures_file, ext='FRAMES')
jura_tiles = Table.read(jura_tiles_file, format='ascii.csv')
jura_exposures = Table.read(jura_exposures_file, format='fits', hdu='EXPOSURES')
# jura_exposures = read_table(jura_exposures_file, ext='EXPOSURES')
jura_frames = Table.read(jura_exposures_file, format='fits', hdu='FRAMES')
# jura_frames = read_table(jura_exposures_file, ext='FRAMES')

## Do some QA on jura

In [3]:
for c in jura_frames.colnames:
    if hasattr(jura_frames[c], 'mask'):
        print(c)

In [4]:
jura_exposures['MJD'].min()

59198.17546592

In [5]:
jura_exposures_bad_rows = list()
bad_columns = list()
for c in jura_exposures.colnames:
    if hasattr(jura_exposures[c], 'mask'):
        print(c)
        bad_columns.append(c)
        jura_exposures_bad_rows.append(np.where(jura_exposures[c].mask)[0])
    elif jura_exposures[c].dtype.kind == 'f' and not np.isfinite(jura_exposures[c]).all():
        print(c)
        bad_columns.append(c)
        jura_exposures_bad_rows.append(np.where(~np.isfinite(jura_exposures[c]))[0])
    else:
        pass
jura_exposures_bad_rows = np.unique(np.hstack(jura_exposures_bad_rows))
jura_exposures[jura_exposures_bad_rows]

TRANSPARENCY_GFA
SEEING_GFA
FIBER_FRACFLUX_GFA
FIBER_FRACFLUX_ELG_GFA
FIBER_FRACFLUX_BGS_GFA
FIBERFAC_GFA
FIBERFAC_ELG_GFA
FIBERFAC_BGS_GFA
EFFTIME_GFA
EFFTIME_DARK_GFA
EFFTIME_BRIGHT_GFA
EFFTIME_BACKUP_GFA


NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes16,bytes19,float64,float64,float64,bytes6,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
20210202,74307,80679,111.0,41.5,59248.23876322,sv1,dark,elgqso,sv1elgqso,566.04931640625,1.3787630796432495,0.0,dark,0.9,1.014901,0.05597689,0.0,0.0,0.16032128036022186,0.04061273857951164,0.11079356074333191,0.11316009083182305,11.229378700256348,15.55074405670166,3.1662378311157227,28.78382110595703,1.346208930015564,1.3787630796432495,1.572113037109375,1.335391935900363,1.3036317825317383,1.6568270921707153,2.038628339767456,0.059126717922222294,0.9789517068362219,0.6320627694913908,0.44970044302469636,0.20504407421980816,--,--,--,1.0143504042229632,21.72197341353745,22.032138412537737,21.185824579206965,19.993767541215583,--,--,--,--
20210308,79769,80731,192.86,27.13,59282.39481033,sv1,other,ssv,sv1ssv,300.06988525390625,5.455439168144949e-05,0.0,other,0.9,1.006417,0.010517629,0.0,0.0,6.3435336414841e-06,1.632539010643086e-06,5.012580004404299e-06,4.612325330949969e-06,0.0004756940761581063,0.0009444231982342899,0.0001949945726664737,0.0017751841805875301,6.09058843110688e-05,5.455439168144949e-05,6.659716746071354e-05,5.442963157261309e-05,7.917178299976513e-05,0.00010203664714936167,0.00012572830019053072,--,1.8761397170576326,0.2613052900585784,0.21108856454109742,0.10411062155477979,--,--,--,1.0072026342866551,21.440068174240203,22.325782890590858,21.242858393524138,19.575426801000127,--,--,--,--
20210404,83420,80697,145.0,32.375,59309.20018021,sv1,dark,elg,sv1elg,80.4530029296875,2.655843309185002e-05,0.0,dark,0.9,1.009287,0.018713951,0.0,9.999999974752427e-07,3.0881897146173287e-06,7.788806897224276e-07,2.608712293294957e-06,1.466542706957119e-06,0.00022730817727278918,0.0005326841492205858,0.00010929051495622844,0.001005950034596026,3.169743649777956e-05,2.655843309185002e-05,3.182314321747981e-05,1.7306537049661673e-05,4.465535675990395e-05,5.718947795685381e-05,7.124691182980314e-05,--,--,--,--,--,0.0007895831162312318,0.0006804336788210475,0.00030002771194605096,1.0096259147044822,20.890481710495784,22.02012195750049,20.896367130054006,19.65692757225773,1.0075802546981036e-05,1.0075802546981036e-05,--,--
20211125,110852,42262,5.991,4.792,59544.11509409,main,backup,backup,mainbackup,601.6654663085938,17.180953979492188,60.0,backup,0.85,1.143367,0.023625372,3.7747349739074707,9.584416389465332,2.200996160507202,0.371442973613739,1.639838457107544,0.8859939349466004,153.8613739013672,200.04493713378906,42.91327667236328,242.5814971923828,19.925031661987305,18.92856788635254,21.540592193603516,10.455533813088758,16.769933700561523,22.45563507080078,17.180953979492188,0.8780836223539111,3.20778365614237,0.11836103479224255,0.10716889904754712,0.057902263978654445,--,--,--,1.1394144832564999,20.885443339512,21.947611923886726,20.879299447705147,19.157658679585744,--,--,--,--
20230815,190752,40203,320.573,26.421,60172.30414529,main,backup,backup,mainbackup,603.0149536132812,0.6969366669654846,60.0,backup,0.85,1.00505,0.113822035,1.1314209699630737,0.5669659972190857,0.05005450174212456,0.

### All exposures for the tiles that contain bad exposures

In [6]:
jura_exposures_bad_tiles = np.in1d(jura_exposures['TILEID'], jura_exposures[jura_exposures_bad_rows]['TILEID'])
jura_exposures[jura_exposures_bad_tiles]

NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes16,bytes19,float64,float64,float64,bytes6,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
20210116,72673,80679,111.0,41.5,59231.12895136,sv1,dark,elgqso,sv1elgqso,1010.651611328125,326.81878662109375,0.0,dark,0.9,1.482236,0.05597689,0.0,0.0,38.00218200683594,9.077220916748047,24.917524337768555,20.483724045877654,2201.41162109375,3279.411865234375,620.1251831054688,4406.3583984375,302.7630310058594,326.81878662109375,308.1976318359375,241.72656372931172,274.91583251953125,324.4987487792969,312.08251953125,0.9257864865818078,1.364426323652585,0.4462013107275403,0.3377375788291352,0.15950337782727625,0.6772962937758491,0.7025498085941242,0.7211024840249856,1.4413451953169076,20.83740392207206,21.55940281058789,20.869991093619294,19.250177355677746,310.7602472291411,310.7602472291411,336.21642213713244,311.79303047402664
20210116,72674,80679,111.0,41.5,59231.14249373,sv1,dark,elgqso,sv1elgqso,1010.6531982421875,228.2066650390625,0.0,dark,0.9,1.388896,0.05597689,0.0,0.0,26.53565788269043,6.325825214385986,17.31275177001953,13.558226335638514,1547.7578125,2282.546142578125,436.15582275390625,3202.080078125,210.3604278564453,228.2066650390625,216.68609619140625,159.99939537545802,191.34774780273438,228.2313690185547,226.78890991210938,0.9475556111630137,1.6302125889740349,0.34066360622529523,0.2679592018499404,0.12973441899483829,0.5306614370188526,0.5729489450180207,0.6026647563042195,1.3545293560233342,21.169436710848696,21.68590126014715,21.019543175271224,19.31885211317393,235.96054893425293,235.96054893425293,262.5946179862971,205.85191926883004
20210116,72675,80679,111.0,41.5,59231.15670561,sv1,dark,elgqso,sv1elgqso,1010.6358642578125,431.2896423339844,0.0,dark,0.9,1.307694,0.05597689,0.0,0.0,50.14995574951172,11.90613842010498,33.162113189697266,28.90362471476611,2921.8876953125,4458.2001953125,833.0277709960938,5540.57763671875,402.9397888183594,431.2896423339844,409.06427001953125,341.08904542814844,373.7346496582031,435.90631103515625,392.4141540527344,0.9695156098579968,1.3579111523188505,0.4527331893708899,0.34222520682290225,0.1614381877960882,0.7185084475845507,0.7457858795252054,0.7659752808486764,1.2792557860094478,21.09543434026307,21.783557681256504,21.112778943284468,19.434942055885493,435.6703368847715,435.6703368847715,456.4805008164956,397.01283958969
20210116,72676,80679,111.0,41.5,59231.17094706,sv1,dark,elgqso,sv1elgqso,1010.640625,309.4156494140625,0.0,dark,0.9,1.240398,0.05597689,0.0,0.0,35.97856140136719,8.54248046875,23.125234603881836,18.996992857505877,2050.183837890625,3034.199951171875,571.170166015625,4036.213134765625,280.9856262207031,309.4156494140625,287.0257263183594,224.18178424734873,254.35951232910156,298.881591796875,285.8667907714844,0.9526606259954222,1.5731332297071055,0.3627174614872012,0.28311219188756376,0.13638338788201682,0.5745298875917932,0.6137477045505411,0.6419004140950773,1.216937400415885,21.471819527753233,21.898186153449608,21.179103001264128,19.503674703231738,314.06639070601557,314.06639070601557,338.52986361519066,264.136996670

In [7]:
jura_tiles_bad_rows = np.in1d(jura_tiles['TILEID'], jura_exposures[jura_exposures_bad_rows]['TILEID'])
jura_tiles[jura_tiles_bad_rows]

TILEID,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,NEXP,EXPTIME,TILERA,TILEDEC,EFFTIME_ETC,EFFTIME_SPEC,EFFTIME_GFA,GOALTIME,OBSSTATUS,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GOALTYPE,MINTFRAC,LASTNIGHT
int64,str7,str6,str16,str19,int64,float64,float64,float64,float64,float64,float64,float64,str8,float64,float64,float64,float64,str6,float64,int64
80679,sv1,dark,elgqso,sv1elgqso,12,7917.0,111.0,41.5,0.0,3884.8,3669.7,4000.0,obsend,3581.6,3884.8,3610.5,3025.0,dark,0.9,20210315
80731,sv1,other,ssv,sv1ssv,21,6663.5,192.86,27.13,0.0,783.7,713.7,1000.0,obsstart,704.1,783.7,768.6,570.6,other,0.9,20210402
80697,sv1,dark,elg,sv1elg,8,6612.3,145.0,32.375,0.0,4499.6,4616.3,4000.0,obsend,4252.1,4499.6,4293.5,5422.0,dark,0.9,20210404
42262,main,backup,backup,mainbackup,3,1454.8,5.991,4.792,51.1,65.8,70.4,60.0,obsend,51.2,57.1,55.9,44.6,backup,0.85,20221222
40203,main,backup,backup,mainbackup,2,920.8,320.573,26.421,72.2,60.5,80.9,60.0,obsend,55.6,54.5,59.7,29.8,backup,0.85,20230913


## Find jura exposures not in daily, daily exposures not in jura

In [8]:
assert (np.unique(jura_exposures['EXPID']) == sorted(jura_exposures['EXPID'])).all()
assert (np.unique(daily_exposures['EXPID']) == sorted(daily_exposures['EXPID'])).all()
assert (np.unique(jura_frames['EXPID']) == sorted(jura_exposures['EXPID'])).all()
assert (np.unique(daily_frames['EXPID']) == sorted(daily_exposures['EXPID'])).all()

In [9]:
first_jura_exposure, last_jura_exposure = jura_exposures['EXPID'].min(), jura_exposures['EXPID'].max()
first_jura_exposure, last_jura_exposure

(67710, 235203)

In [10]:
first_jura_night = jura_exposures['NIGHT'][jura_exposures['EXPID'] == first_jura_exposure].min()
last_jura_night = jura_exposures['NIGHT'][jura_exposures['EXPID'] == last_jura_exposure].max()
first_jura_night, last_jura_night

(20201214, 20240409)

In [11]:
jura_tiles['LASTNIGHT'].min(), jura_tiles['LASTNIGHT'].max()

(20201216, 20240409)

In [12]:
jura_expid_set = frozenset(jura_exposures['EXPID'].tolist())
daily_expid_set = frozenset(daily_exposures['EXPID'].tolist())

In [13]:
jura_not_in_daily = jura_expid_set - daily_expid_set
jura_not_in_daily

frozenset({80478, 80681, 80688, 80691, 82603, 82622, 82625, 221977})

In [14]:
daily_not_in_jura = daily_expid_set - jura_expid_set
# daily_not_in_jura

## Patch frames

We don't necessarily want to change the values in the `FRAMES` table, just make sure it is consistent with the `EXPOSURES` table.

In [15]:
daily_frames_join = Table()
daily_frames_join['FRAMEID'] = np.array([100*row['EXPID'] + cameraid(row['CAMERA']) for row in daily_frames])
daily_frames_join['DAILY_INDEX'] = np.arange(len(daily_frames))

In [16]:
jura_frames_join = Table()
jura_frames_join['FRAMEID'] = np.array([100*row['EXPID'] + cameraid(row['CAMERA']) for row in jura_frames])
jura_frames_join['JURA_INDEX'] = np.arange(len(jura_frames))

In [17]:
j = join(daily_frames_join, jura_frames_join, join_type='outer', keys='FRAMEID')
# j

In [18]:
daily_frames_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['DAILY_INDEX']
jura_frames_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['JURA_INDEX']

In [19]:
daily_frames_patched = daily_frames.copy()
for column in daily_frames_patched.colnames:
    if hasattr(daily_frames_patched[column], 'mask') and not column.startswith('TSNR2_'):
        print(f"Patching {column}...")
        daily_frames_patched[column][daily_frames_index] = jura_frames[column][jura_frames_index]
        daily_frames_patched[column].mask[daily_frames_index] = False
# daily_frames_patched

Patching SEEING_ETC...
Patching EBV...


## Patch exposures

We want to only *patch* and rows that:

* Appear in `jura`.
* Have `NIGHT >= first_jura_night`.
* Have `EFFTIME_SPEC > 0`.

We want to only *load* rows that:

* Have `NIGHT >= first_jura_night`.
* Have `EFFTIME_SPEC > 0`.

which is slightly different. However we don't want to *remove* rows that *don't* satisfy these criteria.

In [20]:
daily_exposures_join = Table()
daily_exposures_join['EXPID'] = daily_exposures['EXPID']
# daily_exposures_join['NIGHT'] = daily_exposures['NIGHT']
# daily_exposures_join['EFFTIME_SPEC'] = daily_exposures['EFFTIME_SPEC']
daily_exposures_join['DAILY_INDEX'] = np.arange(len(daily_exposures))

In [21]:
jura_exposures_join = Table()
jura_exposures_join['EXPID'] = jura_exposures['EXPID']
jura_exposures_join['JURA_INDEX'] = np.arange(len(jura_exposures))

In [22]:
j = join(daily_exposures_join, jura_exposures_join, join_type='outer', keys='EXPID')
j

EXPID,DAILY_INDEX,JURA_INDEX
int32,int64,int64
50986,0,--
50988,1,--
50991,2,--
50995,3,--
51001,4,--
51002,5,--
51028,6,--
51029,7,--
51030,8,--


In [23]:
daily_exposures_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['DAILY_INDEX']
jura_exposures_index = j[(~j['JURA_INDEX'].mask) & (~j['DAILY_INDEX'].mask)]['JURA_INDEX']

In [24]:
daily_exposures_patched = daily_exposures.copy()
can_patch = ('NIGHT', 'EXPID', 'TILEID', 'TILERA', 'TILEDEC', 'MJD', 'SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'EXPTIME', 'GOALTIME', 'GOALTYPE', 'MINTFRAC', 'AIRMASS', 'EBV', 'SEEING_ETC', 'EFFTIME_ETC',
             'TRANSPARENCY_GFA', 'SEEING_GFA', 'FIBER_FRACFLUX_GFA', 'FIBER_FRACFLUX_ELG_GFA', 'FIBER_FRACFLUX_BGS_GFA', 'FIBERFAC_GFA', 'FIBERFAC_ELG_GFA', 'FIBERFAC_BGS_GFA', 'AIRMASS_GFA', 'SKY_MAG_AB_GFA',
             'EFFTIME_GFA', 'EFFTIME_DARK_GFA', 'EFFTIME_BRIGHT_GFA', 'EFFTIME_BACKUP_GFA')
for column in ['TILERA', 'TILEDEC', 'MJD', 'SURVEY'] + [c for c in daily_exposures_patched.colnames if hasattr(daily_exposures_patched[c], 'mask') and c in can_patch]:
    print(f"Patching {column}...")
    if hasattr(jura_exposures[column], 'mask'):
        if np.any(jura_exposures[column].mask[jura_exposures_index]):
            jura_exposures[column][jura_exposures[column].mask] = 0
            jura_exposures[column].mask[jura_exposures[column].mask] = False
    daily_exposures_patched[column][daily_exposures_index] = jura_exposures[column][jura_exposures_index]
    if hasattr(daily_exposures_patched[column], 'mask'):
        daily_exposures_patched[column].mask[daily_exposures_index] = False

known_bad_exposures = np.in1d(daily_exposures_patched['EXPID'], np.array([74307, 79769, 83420, 110852, 190752]))
daily_exposures_patched[known_bad_exposures]

Patching TILERA...
Patching TILEDEC...
Patching MJD...
Patching SURVEY...
Patching EBV...
Patching SEEING_ETC...
Patching TRANSPARENCY_GFA...
Patching SEEING_GFA...
Patching FIBER_FRACFLUX_GFA...
Patching FIBER_FRACFLUX_ELG_GFA...
Patching FIBER_FRACFLUX_BGS_GFA...
Patching FIBERFAC_GFA...
Patching FIBERFAC_ELG_GFA...
Patching FIBERFAC_BGS_GFA...
Patching EFFTIME_GFA...
Patching EFFTIME_DARK_GFA...
Patching EFFTIME_BRIGHT_GFA...
Patching EFFTIME_BACKUP_GFA...


NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes19,bytes19,float64,float64,float64,bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
20210202,74307,80679,111.0,41.5,59248.23876322,sv1,dark,elgqso,sv1elgqso,566.04931640625,1.4423755109310155,4000.0,dark,0.9,1.014901,0.05597689002752304,0.0,0.0,0.1677180826663971,0.04438820853829384,0.11977556347846985,0.1674174666404724,11.726827621459961,0.0,0.0,0.0,1.455345647106182,1.4423755109310155,1.6417558670043948,1.9152153084460037,0.0,0.0,0.0,0.059126717922222294,0.9789517068362219,0.6320627694913908,0.44970044302469636,0.20504407421980816,0.0,0.0,0.0,1.0143504042229632,21.72197341353745,21.95553631921876,21.212220164279294,20.113056586954166,0.0,0.0,0.0,0.0
20210308,79769,80731,192.86,27.13,59282.39481033,sv1,other,ssv,sv1ssv,300.06988525390625,4.908081118628617e-05,1000.0,unknown,0.9,1.006417,0.010517628863453865,0.0,0.0,5.707071068172809e-06,1.5589646409353008e-06,3.6969945540477056e-06,6.8609160734922625e-06,0.0003828057087957859,0.0,0.0,0.0,4.4920723187211217e-05,4.908081118628617e-05,5.3592799231410035e-05,7.848721974830708e-05,0.0,0.0,0.0,0.0,1.8761397170576326,0.2613052900585784,0.21108856454109742,0.10411062155477979,0.0,0.0,0.0,1.0072026342866551,21.440068174240203,22.274960404546178,21.280958712336684,19.68924689265133,0.0,0.0,0.0,0.0
20210404,83420,80697,145.0,32.375,59309.20018021,sv1,dark,elg,sv1elg,80.4530029296875,4.205021150482936e-05,4000.0,dark,0.9,1.0092869997024536,0.018713951110839844,0.0,0.0,4.889559477305738e-06,9.873140243144007e-07,4.668986548495013e-06,6.542905478545882e-07,0.00034244905691593885,0.0,0.0,0.0,5.67310146778893e-05,4.205021150482936e-05,4.7942867968231444e-05,7.484925549098573e-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0007895831162312318,0.0006804336788210475,0.00030002771194605096,1.0096259147044822,20.890481710495784,21.92729853866057,20.87627884543552,19.709699199758923,1.0075802546981036e-05,1.0075802546981036e-05,0.0,0.0
20211125,110852,42262,5.991,4.792,59544.11509409,main,backup,backup,mainbackup,601.6654663085938,17.011510848999023,60.0,backup,0.85,1.143367052078247,0.023625371977686882,3.7747349739074707,9.584416389465332,2.1469082832336426,0.3637787997722626,1.6068999767303467,0.8854541587585117,150.91136169433594,197.9574432373047,42.43981170654297,240.1890869140625,19.524808883666992,18.463411331176758,21.12759017944336,10.449163963405304,16.594938278198242,22.2078800201416,17.011510848999023,0.8780836223539111,3.20778365614237,0.11836103479224255,0.10716889904754712,0.057902263978654445,0.0,0.0,0.0,1.1394144832564999,20.885443339512,21.82595186409396,20.871110333724605,19.155375680908563,0.0,0.0,0.0,0.0
20230815,190752,40203,320.573,26.421,60172.30414529,main,backup,backup,mainbackup,603.0149536132812,0.6952639818191528,60.0,backup,0.85,1.0050499439239502,0.11382203549146652,1.1314209699630737,0.5669659972190857,0.04952569305896759,0.01299299392849207,0.03662736713886261,0.048888119613252456,3.586458444595337,5.241128921508789,1.040251612663269,9.816577911376953,0.44504472613334656,0.4259209930896759,0.5021041631698608,0.5769242514120

### After patching are there still missing data?

In [25]:
assert not (daily_exposures_patched['TILERA'] == daily_exposures['TILERA']).all()
assert not (daily_exposures_patched['TILEDEC'] == daily_exposures['TILEDEC']).all()
assert not (daily_exposures_patched['MJD'] == daily_exposures['MJD']).all()
assert not (daily_exposures_patched['SURVEY'] == daily_exposures['SURVEY']).all()
assert (daily_exposures_patched['PROGRAM'] == daily_exposures['PROGRAM']).all()
assert (daily_exposures_patched['FAPRGRM'] == daily_exposures['FAPRGRM']).all()
assert (daily_exposures_patched['FAFLAVOR'] == daily_exposures['FAFLAVOR']).all()

In [26]:
missing_mjd = np.where((daily_exposures_patched['NIGHT'] >= first_jura_night) & (daily_exposures_patched['EFFTIME_SPEC'] > 0) & (daily_exposures_patched['MJD'] < 50000))[0]
for row in daily_exposures_patched[missing_mjd]:
    raw_data_file = os.path.join(os.environ['DESI_SPECTRO_DATA'], "{0:08d}".format(row['NIGHT']), "{0:08d}".format(row['EXPID']), "desi-{0:08d}.fits.fz".format(row['EXPID']))
    with fits.open(raw_data_file, mode='readonly') as hdulist:
        mjd_obs = hdulist['SPEC'].header['MJD-OBS']
    print("INFO: tile {0:d} exposure {1:d} has MJD-OBS = {2:f} in {3}!".format(row['TILEID'], row['EXPID'], mjd_obs, raw_data_file))
    w = np.where(daily_exposures_patched['EXPID'] == row['EXPID'])[0]
    assert len(w) == 1
    daily_exposures_patched['MJD'][w] = mjd_obs
    print(daily_exposures_patched[w][['NIGHT', 'EXPID', 'TILEID', 'MJD']])

INFO: tile 80618 exposure 69082 has MJD-OBS = 59204.557878 in /global/cfs/cdirs/desi/spectro/data/20201220/00069082/desi-00069082.fits.fz!
 NIGHT   EXPID TILEID      MJD      
-------- ----- ------ --------------
20201220 69082  80618 59204.55787815
INFO: tile 80618 exposure 69083 has MJD-OBS = 59204.563137 in /global/cfs/cdirs/desi/spectro/data/20201220/00069083/desi-00069083.fits.fz!
 NIGHT   EXPID TILEID      MJD      
-------- ----- ------ --------------
20201220 69083  80618 59204.56313729
INFO: tile 80618 exposure 69084 has MJD-OBS = 59204.568494 in /global/cfs/cdirs/desi/spectro/data/20201220/00069084/desi-00069084.fits.fz!
 NIGHT   EXPID TILEID      MJD      
-------- ----- ------ --------------
20201220 69084  80618 59204.56849373
INFO: tile 80618 exposure 69085 has MJD-OBS = 59204.573527 in /global/cfs/cdirs/desi/spectro/data/20201220/00069085/desi-00069085.fits.fz!
 NIGHT   EXPID TILEID      MJD      
-------- ----- ------ --------------
20201220 69085  80618 59204.57352664


In [27]:
still_missing_mjd = np.where((daily_exposures_patched['NIGHT'] >= first_jura_night) & (daily_exposures_patched['EFFTIME_SPEC'] > 0) & (daily_exposures_patched['MJD'] < 50000))[0]
daily_exposures_patched[still_missing_mjd]

NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes19,bytes19,float64,float64,float64,bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


In [28]:
possible_expid_still_patchable = list()
for c in daily_exposures_patched.colnames:
    if hasattr(daily_exposures_patched[c], 'mask'):
        if daily_exposures_patched[c].mask.any():
            n_masked = daily_exposures_patched[c].mask.sum()
            not_in_jura = [e in daily_not_in_jura for e in daily_exposures_patched['EXPID'][daily_exposures_patched[c].mask]]
            if all(not_in_jura):
                print(f"Column {c} still has {n_masked:d} masked values, but the exposures are not in jura and cannot be patched.")
            else:
                if c in can_patch:
                    for e in daily_exposures_patched['EXPID'][daily_exposures_patched[c].mask]:
                        if e not in daily_not_in_jura:
                            possible_expid_still_patchable.append(int(e))
                    print(f"Column {c} still has {n_masked:d} masked values, and some exposures can still be patched.")
                else:
                    print(f"Column {c} still has {n_masked:d} masked values, and some exposures could still be patched, but they do not meet the patchable column guidelines.")

possible_expid_still_patchable = np.unique(np.array(possible_expid_still_patchable))
daily_exposures_patched[np.in1d(daily_exposures_patched['EXPID'], possible_expid_still_patchable)]

Column EFFTIME_SPEC still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column EBV still has 12 masked values, but the exposures are not in jura and cannot be patched.
Column SEEING_ETC still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_ELG still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_QSO still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_LRG still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_LYA still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_BGS still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_GPBDARK still has 1 masked values, but the exposures are not in jura and cannot be patched.
Column TSNR2_GPBBRIGHT still has 1 masked values, but the exposures are not in jura and 

NIGHT,EXPID,TILEID,TILERA,TILEDEC,MJD,SURVEY,PROGRAM,FAPRGRM,FAFLAVOR,EXPTIME,EFFTIME_SPEC,GOALTIME,GOALTYPE,MINTFRAC,AIRMASS,EBV,SEEING_ETC,EFFTIME_ETC,TSNR2_ELG,TSNR2_QSO,TSNR2_LRG,TSNR2_LYA,TSNR2_BGS,TSNR2_GPBDARK,TSNR2_GPBBRIGHT,TSNR2_GPBBACKUP,LRG_EFFTIME_DARK,ELG_EFFTIME_DARK,BGS_EFFTIME_BRIGHT,LYA_EFFTIME_DARK,GPB_EFFTIME_DARK,GPB_EFFTIME_BRIGHT,GPB_EFFTIME_BACKUP,TRANSPARENCY_GFA,SEEING_GFA,FIBER_FRACFLUX_GFA,FIBER_FRACFLUX_ELG_GFA,FIBER_FRACFLUX_BGS_GFA,FIBERFAC_GFA,FIBERFAC_ELG_GFA,FIBERFAC_BGS_GFA,AIRMASS_GFA,SKY_MAG_AB_GFA,SKY_MAG_G_SPEC,SKY_MAG_R_SPEC,SKY_MAG_Z_SPEC,EFFTIME_GFA,EFFTIME_DARK_GFA,EFFTIME_BRIGHT_GFA,EFFTIME_BACKUP_GFA
int32,int32,int32,float64,float64,float64,bytes7,bytes6,bytes19,bytes19,float64,float64,float64,bytes7,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


### Fill remaining masked values with zero.

In [29]:
for c in daily_exposures_patched.colnames:
    if hasattr(daily_exposures_patched[c], 'mask'):
        if daily_exposures_patched[c].mask.any():
            daily_exposures_patched[c][daily_exposures_patched[c].mask] = 0
            daily_exposures_patched[c].mask[daily_exposures_patched[c].mask] = False

## Patch tiles

Similar to the discussion above, we want `LASTNIGHT >= first_jura_night`.

In [30]:
assert (np.unique(jura_exposures['TILEID']) == sorted(jura_tiles['TILEID'])).all()
assert (np.unique(daily_exposures['TILEID']) == sorted(daily_tiles['TILEID'])).all()

### First patch PROGRAM with faflavor2program()

In [31]:
daily_tiles_patched = daily_tiles.copy()
daily_tiles_patched['PROGRAM'] = faflavor2program(daily_tiles_patched['FAFLAVOR'])

In [34]:
oddball_survey = np.where((daily_tiles_patched['SURVEY'] != 'cmx') & (daily_tiles_patched['SURVEY'] != 'sv1') & (daily_tiles_patched['SURVEY'] != 'sv2') & (daily_tiles_patched['SURVEY'] != 'sv3') & (daily_tiles_patched['SURVEY'] != 'main') & (daily_tiles_patched['SURVEY'] != 'special'))[0]
oddball_program = np.where((daily_tiles_patched['PROGRAM'] != 'backup') & (daily_tiles_patched['PROGRAM'] != 'bright') & (daily_tiles_patched['PROGRAM'] != 'dark') & (daily_tiles_patched['PROGRAM'] != 'other'))[0]

In [35]:
assert (daily_tiles['SURVEY'][oddball_survey] == 'unknown').all()
assert len(oddball_program) == 0

In [36]:
unknown_cmx = np.where((daily_tiles['SURVEY'] == 'unknown') & (daily_tiles['FAFLAVOR'] != 'unknown'))[0]
print(daily_tiles[['TILEID', 'SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE', 'LASTNIGHT']][unknown_cmx])

TILEID  SURVEY PROGRAM  FAPRGRM   FAFLAVOR GOALTYPE LASTNIGHT
------ ------- ------- --------- --------- -------- ---------
 80462 unknown      --  dithprec  dithprec  unknown  20210103
 80345 unknown      --  dithprec  dithprec  unknown  20210103
 80769 unknown      -- dithfocus dithfocus  unknown  20210219
 80804 unknown      --  dithprec  dithprec  unknown  20210224
 80743 unknown      --  dithprec  dithprec  unknown  20210305
 80916 unknown      -- dithfocus dithfocus  unknown  20210422
 82052 unknown      -- dithfocus dithfocus  unknown  20210423
 82026 unknown      --  dithprec  dithprec  unknown  20210423
 82065 unknown      -- dithfocus dithfocus  unknown  20210423
 82039 unknown      --  dithprec  dithprec  unknown  20210529


In [37]:
unknown_unknown = np.where((daily_tiles['SURVEY'] == 'unknown') & (daily_tiles['FAFLAVOR'] == 'unknown'))[0]
daily_tiles['LASTNIGHT'][unknown_unknown].max()

20200315

In [38]:
print(daily_tiles[['TILEID', 'SURVEY', 'PROGRAM', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE', 'LASTNIGHT']][unknown_unknown])

TILEID  SURVEY PROGRAM FAPRGRM FAFLAVOR GOALTYPE LASTNIGHT
------ ------- ------- ------- -------- -------- ---------
 70004 unknown      -- unknown  unknown  unknown  20200219
 70508 unknown      -- unknown  unknown  unknown  20200225
 70506 unknown      -- unknown  unknown  unknown  20200225
 70512 unknown      -- unknown  unknown  unknown  20200226
 70514 unknown      -- unknown  unknown  unknown  20200227
 70502 unknown      -- unknown  unknown  unknown  20200227
 70513 unknown      -- unknown  unknown  unknown  20200229
 70500 unknown      -- unknown  unknown  unknown  20200303
 70005 unknown      -- unknown  unknown  unknown  20200303
 70510 unknown      -- unknown  unknown  unknown  20200304
   ...     ...     ...     ...      ...      ...       ...
 63234 unknown      -- unknown  unknown  unknown  20200315
 67230 unknown      -- unknown  unknown  unknown  20200315
 63231 unknown      -- unknown  unknown  unknown  20200315
 63228 unknown      -- unknown  unknown  unknown  202003

In [39]:
# For now just patch with 'cmx'
daily_tiles['SURVEY'][oddball_survey] = 'cmx'

## Write out the patched files

In [40]:
daily_tiles_patched.write(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'tiles-daily-patched-with-jura.csv'), format='ascii.csv', overwrite=True)

In [41]:
daily_exposures_patched.write(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'exposures-daily-patched-with-jura.csv'), format='ascii.csv', overwrite=True)

In [42]:
daily_exposures_fits = fits.HDUList([fits.PrimaryHDU(), fits.table_to_hdu(daily_exposures_patched), fits.table_to_hdu(daily_frames_patched)])

In [43]:
daily_exposures_fits.info()

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1  EXPOSURES     1 BinTableHDU    111   23912R x 51C   ['J', 'J', 'J', 'D', 'D', 'D', '7A', '6A', '19A', '19A', 'D', 'D', 'D', '7A', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D']   
  2  FRAMES        1 BinTableHDU     63   703945R x 27C   ['J', 'J', 'J', 'E', 'D', 'D', '2A', 'D', 'D', 'D', 'D', '7A', '7A', '19A', 'D', 'D', 'D', 'D', '19A', 'E', 'E', 'D', 'D', 'D', 'D', 'D', 'D']   


In [44]:
daily_exposures_fits.writeto(os.path.join(os.environ['DESI_ROOT'], 'users', os.environ['USER'], 'exposures-daily-patched-with-jura.fits'), overwrite=True)